# test qwen

In [1]:
from transformers import pipeline

model_name_or_path = "Qwen/Qwen3-8B"

generator = pipeline(
    "text-generation", 
    model_name_or_path, 
    torch_dtype="auto", 
    device_map="auto",
)

/root/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00, 86.65it/s]
Device set to use cpu


In [2]:
generator.model.device

device(type='cpu')

In [3]:
def build_prompt(instruction: str, input_: str | None):
    return f"""You are a classifier.

You MUST output ONLY one of the following labels:
Exact
Substitute
Complement
Irrelevant

Do NOT explain.
Do NOT output anything else.
Do NOT repeat the label.

Your output MUST be exactly one word from the list above.

### TASK
Classify the product's relevance to the query.

### INPUT
{input_}

### OUTPUT"""

In [ ]:
from transformers import pipeline
import json


with open('test_pretrain_model.jsonl', "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        row = json.loads(line)
        prompt = build_prompt(row["instruction"], row.get("input", ""))

        out = generator(
            prompt,
            max_new_tokens=256,
            do_sample=False
        )[0]["generated_text"]

        model_output = out[len(prompt):].strip()

        output = {
            "instruction": row["instruction"],
            "input": row.get("input", ""),
            "reference_output": row["output"],
            "model_output": model_output
        }
        
        with open('qwen_test_100.jsonl', 'a', encoding='utf-8') as wf:
            wf.write(json.dumps(output, ensure_ascii=False) + '\n')
            print('saved up to:', idx)

saved up to: 0
saved up to: 1
saved up to: 2
saved up to: 3
saved up to: 4
saved up to: 5
saved up to: 6
saved up to: 7
saved up to: 8
saved up to: 9
saved up to: 10
saved up to: 11
saved up to: 12
saved up to: 13
saved up to: 14
saved up to: 15
saved up to: 16
saved up to: 17
saved up to: 18
saved up to: 19
saved up to: 20
saved up to: 21
saved up to: 22
saved up to: 23
saved up to: 24
saved up to: 25
saved up to: 26
saved up to: 27
saved up to: 28
saved up to: 29
saved up to: 30
saved up to: 31
saved up to: 32
saved up to: 33
saved up to: 34
saved up to: 35
saved up to: 36
saved up to: 37
saved up to: 38
saved up to: 39
saved up to: 40
saved up to: 41
saved up to: 42
saved up to: 43
saved up to: 44
saved up to: 45
saved up to: 46
saved up to: 47
saved up to: 48
saved up to: 49
saved up to: 50
saved up to: 51
saved up to: 52
saved up to: 53
saved up to: 54
saved up to: 55
saved up to: 56
saved up to: 57
saved up to: 58
saved up to: 59
saved up to: 60
saved up to: 61
saved up to: 62
sa

# qwen result

In [1]:
import json
from sklearn.metrics import f1_score

VALID_LABELS = {"Exact", "Substitute", "Complement", "Irrelevant"}

y_true = []
y_pred = []

with open("qwen_test_100.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)

        ref = item["reference_output"].strip()
        model_out = item["model_output"].strip()

        # 提取第一个词
        first_word = model_out.split()[0]

        # 可选：防止异常输出污染评估
        if first_word not in VALID_LABELS:
            continue   # 或者标记为 Irrelevant / skip

        y_true.append(ref)
        y_pred.append(first_word)

# 计算 micro F1
micro_f1 = f1_score(y_true, y_pred, average="micro")

print(f"Micro F1: {micro_f1:.4f}")
print(f"Evaluated samples: {len(y_true)}")


Micro F1: 0.3600
Evaluated samples: 100


# test mistral

In [1]:
from transformers import pipeline

model_name_or_path = "mistralai/Ministral-3-8B-Instruct-2512"

generator = pipeline(
    "text-generation", 
    model_name_or_path, 
    torch_dtype="auto", 
    device_map="auto",
)

/root/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Using FP8 quantized models requires a GPU or XPU, we will default to dequantizing the model to bf16 since no GPU or XPU is available
Loading weights: 100%|██████████| 531/531 [00:08<00:00, 63.79it/s, Materializing param=model.vision_tower.transformer.layers.23.ffn_norm.weight]               
The model 'Mistral3ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCa

In [4]:
from transformers import pipeline
import json


with open('test_pretrain_model.jsonl', "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        row = json.loads(line)
        prompt = build_prompt(row["instruction"], row.get("input", ""))

        out = generator(
            prompt,
            max_new_tokens=256,
            do_sample=False
        )[0]["generated_text"]

        model_output = out[len(prompt):].strip()

        output = {
            "instruction": row["instruction"],
            "input": row.get("input", ""),
            "reference_output": row["output"],
            "model_output": model_output
        }
        
        with open('mistral_test_100.jsonl', 'a', encoding='utf-8') as wf:
            wf.write(json.dumps(output, ensure_ascii=False) + '\n')
            print('saved up to:', idx)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


saved up to: 0


Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


saved up to: 1


Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

saved up to: 96


Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


saved up to: 97


Both `max_new_tokens` (=256) and `max_length`(=262144) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


saved up to: 98
saved up to: 99


In [5]:
import json
from sklearn.metrics import f1_score

VALID_LABELS = {"Exact", "Substitute", "Complement", "Irrelevant"}

y_true = []
y_pred = []

with open("mistral_test_100.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)

        ref = item["reference_output"].strip()
        model_out = item["model_output"].strip()

        # 提取第一个词
        first_word = model_out.split()[0]

        # 可选：防止异常输出污染评估
        if first_word not in VALID_LABELS:
            continue   # 或者标记为 Irrelevant / skip

        y_true.append(ref)
        y_pred.append(first_word)

# 计算 micro F1
micro_f1 = f1_score(y_true, y_pred, average="micro")

print(f"Micro F1: {micro_f1:.4f}")
print(f"Evaluated samples: {len(y_true)}")


Micro F1: 0.3300
Evaluated samples: 100


# random

In [6]:
import json
import random
from sklearn.metrics import f1_score

LABELS = ["Exact", "Substitute", "Complement", "Irrelevant"]

y_true = []
y_pred = []

random.seed(42)  # 保证可复现

with open("qwen_test_100.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)

        ref = item["reference_output"].strip()
        pred = random.choice(LABELS)

        y_true.append(ref)
        y_pred.append(pred)

micro_f1 = f1_score(y_true, y_pred, average="micro")

print(f"Random (uniform) Micro F1: {micro_f1:.4f}")
print(f"Samples: {len(y_true)}")

Random (uniform) Micro F1: 0.2900
Samples: 100
